# Data Quality Check Script: Explained

**Purpose:** This script acts as a "safety scanner" for our raw AACT text files before we load them into our analysis pipeline. It ensures we don't accidentally lose data due to formatting errors or messy content.

**What it does (Step-by-Step):**

1.  **Iterates through Files:** It automatically finds every `.txt` file in our `00_data` folder.

2.  **Structural Validation (The "Pipe Check"):**
    * It scans every line to count the pipe separators (`|`).
    * *Why?* If a line has too many pipes (e.g., a pipe accidentally typed in a description), it shifts all the data, ruining the row. This script catches those errors.

3.  **Correct Loading:**
    * It attempts to load the file using the "Golden Settings":
        * `sep='|'`: Pipe delimiter.
        * `quotechar='"'`: Respects quotes (fixing the "broken rows" issue we found earlier).
        * `dtype=str`: Reads everything as text first (safest method).

4.  **Content Analysis:**
    * **Whitespace:** Checks if columns have invisible spaces (e.g., `" Diabetes "` vs `"Diabetes"`).
    * **Mixed Types:** Checks if numeric columns contain text (e.g., `"1,200"` or `"18-35"`).

5.  **Reporting:**
    * It saves a full audit log to `data_quality_report_v2.txt` so we have a permanent record of our data health.

**Why use it?**
Running this ensures our cleaning script (`clean_aact_final.py`) is based on *facts*, not guesses. It proves our data integrity is solid before we start Machine Learning.

In [ ]:
import pandas as pd
import csv
import sys
import os
import re
from datetime import datetime

# --- CONFIGURATION ---
DATA_DIR = "00_data"
REPORT_FILENAME = "data_quality_report_v2.txt" # New report name to distinguish from previous run
DELIMITER = "|"
EXTENSION_TO_CHECK = ".txt"

# Buffer to hold report text for file writing
report_buffer = []

def log(message):
    """Helper function to print to console AND buffer for the report file."""
    print(message)
    report_buffer.append(message)

def save_report():
    """Writes the buffered log to the text file."""
    report_path = REPORT_FILENAME
    try:
        with open(report_path, "w", encoding="utf-8") as f:
            f.write("\n".join(report_buffer))
        print(f"\n[SUCCESS] Full report saved to: {os.path.abspath(report_path)}")
    except Exception as e:
        print(f"\n[ERROR] Could not save report file: {e}")

# --- THE CORRECT LOADING FUNCTION ---
def load_dataframe_safely(file_path, delimiter):
    """
    Loads a pipe-delimited file using the correct settings to handle quotes.
    Returns: DataFrame or None (if failed)
    """
    try:
        df = pd.read_csv(
            file_path,
            sep=delimiter,
            dtype=str,                  # 1. Read as text (prevent type errors)
            low_memory=False,           # 2. Read full file at once (prevent chunking errors)
            quotechar='"',              # 3. Handle quotes properly (fixes "broken rows")
            quoting=csv.QUOTE_MINIMAL,  # 4. Allow quotes to wrap fields containing pipes
            on_bad_lines='warn'         # 5. Warn if genuine corruption exists
        )
        return df
    except Exception as e:
        log(f"  [Critical Load Error] {e}")
        return None

def check_whitespace_issues(df):
    """Finds and lists columns that need trimming."""
    log("\n--- 1. HYGIENE: Whitespace Check ---")
    text_cols = df.select_dtypes(include=['object']).columns
    dirty_cols = []

    for col in text_cols:
        sample = df[col].dropna().head(5000).astype(str)
        if sample.empty: continue
        if (sample.str.len() != sample.str.strip().str.len()).any():
            dirty_cols.append(col)

    if dirty_cols:
        log(f"  [Action Needed] {len(dirty_cols)} columns have invisible spaces (e.g. ' Diabetes ').")
        log(f"  Columns: {dirty_cols[:5]}...")
    else:
        log("  [OK] No whitespace issues found.")

def check_mixed_types(df):
    """Checks if a column has both numbers and text (dangerous for analysis)."""
    log("\n--- 2. INTEGRITY: Mixed Data Types ---")

    for col in df.columns:
        # Skip if explicitly string
        if df[col].dtype == 'object':
            # Try to force numeric
            numeric_vals = pd.to_numeric(df[col], errors='coerce')
            num_count = numeric_vals.notna().sum()
            total_count = df[col].notna().sum()

            # If a column is mostly numbers (90%) but has some text, warn user
            if total_count > 0 and (num_count / total_count) > 0.9 and (num_count != total_count):
                log(f"  [Warning] Column '{col}' looks numeric but has {total_count - num_count} text values.")
                # Show non-numeric examples
                non_nums = df[col][numeric_vals.isna() & df[col].notna()]
                log(f"    Garbage examples: {non_nums.head(3).tolist()}")

def check_structure_and_counts(df, file_path):
    """Basic structure check using the DF shape (since we trust the loader now)."""
    log(f"\n--- 3. STRUCTURE: Row/Col Count ---")
    rows, cols = df.shape
    log(f"  Loaded {rows} rows and {cols} columns.")

    # Check for empty columns
    empty_cols = [col for col in df.columns if df[col].isnull().all()]
    if empty_cols:
        log(f"  [Info] {len(empty_cols)} columns are 100% EMPTY.")

def process_file(file_name):
    # Construct full path correctly for Linux/WSL
    file_path = os.path.join(DATA_DIR, file_name)

    log("\n" + "="*60)
    log(f"PROCESSING: {file_path}")
    log("="*60)

    if not os.path.exists(file_path):
        log(f"Error: File not found at {file_path}")
        return

    # Use the CORRECT loader
    df = load_dataframe_safely(file_path, DELIMITER)

    if df is not None:
        check_structure_and_counts(df, file_path)
        check_whitespace_issues(df)
        check_mixed_types(df)

# --- EXECUTION LOOP ---
log(f"Starting FINAL Data Quality Scan at {datetime.now()}")

if not os.path.exists(DATA_DIR):
    log(f"CRITICAL ERROR: The folder '{DATA_DIR}' was not found.")
else:
    all_files = os.listdir(DATA_DIR)
    target_files = [f for f in all_files if f.endswith(EXTENSION_TO_CHECK)]

    if not target_files:
        log(f"No files ending with '{EXTENSION_TO_CHECK}' found in '{DATA_DIR}'.")
    else:
        log(f"Found {len(target_files)} files in '{DATA_DIR}'.")
        for file_name in target_files:
            process_file(file_name)

# Finally, save the report to disk
save_report()

Starting FINAL Data Quality Scan at 2025-11-28 22:26:02.342689
Found 50 files in '00_data'.

PROCESSING: 00_data/calculated_values.txt

--- 3. STRUCTURE: Row/Col Count ---
  Loaded 558973 rows and 19 columns.
  [Info] 1 columns are 100% EMPTY.

--- 1. HYGIENE: Whitespace Check ---
  [OK] No whitespace issues found.

--- 2. INTEGRITY: Mixed Data Types ---

PROCESSING: 00_data/responsible_parties.txt

--- 3. STRUCTURE: Row/Col Count ---
  Loaded 540493 rows and 8 columns.

--- 1. HYGIENE: Whitespace Check ---
  [OK] No whitespace issues found.

--- 2. INTEGRITY: Mixed Data Types ---

PROCESSING: 00_data/drop_withdrawals.txt

--- 3. STRUCTURE: Row/Col Count ---
  Loaded 563769 rows and 10 columns.
  [Info] 3 columns are 100% EMPTY.

--- 1. HYGIENE: Whitespace Check ---
  [OK] No whitespace issues found.

--- 2. INTEGRITY: Mixed Data Types ---

PROCESSING: 00_data/countries.txt

--- 3. STRUCTURE: Row/Col Count ---
  Loaded 763540 rows and 4 columns.

--- 1. HYGIENE: Whitespace Check ---
  

# 🕵️ Deep Dive Diagnostic Script: Explained

**Purpose:** This script is a targeted diagnostic tool that investigates specific issues flagged by the initial `check_data_quality.py` report. It drills down into problematic files to reveal exactly *why* rows are broken or why data types are mixed, helping us design the perfect cleaning strategy.

**What it does (Step-by-Step):**

1.  **Structural Investigator ("Broken Row Finder"):**
    * **Target:** Files flagged with broken rows (e.g., `responsible_parties.txt`, `facilities.txt`).
    * **Action:** It reads the specific lines where column counts don't match the header.
    * **Output:** It extracts the raw text of these broken lines so we can see the root cause (e.g., a pipe `|` character hidden inside a job title like `"Doctor | Professor"`).
    * **Benefit:** Confirms if we need to change our loading parameters (e.g., enabling `quotechar='"'`).

2.  **Mixed Type Investigator ("Hidden String Finder"):**
    * **Target:** Numeric columns that contain text (e.g., `param_value` in `baseline_measurements.txt`).
    * **Action:** It tries to convert the column to numbers and isolates the values that fail.
    * **Output:** It lists the most frequent non-numeric patterns.
        * *Example Findings:* It distinguishes between fixable issues (like `"10,366"` or `"- 4.5"`) vs. complex data (like `"18-35"` ranges) vs. pure garbage (like `"units"`).
    * **Benefit:** Tells us exactly which cleaning functions to write (e.g., "strip commas", "remove spaces").

3.  **Unparseable Date Finder:**
    * **Target:** Date columns with errors (e.g., `anticipated_posting_date`).
    * **Action:** It finds values that fail standard datetime conversion or fall outside a reasonable year range (1900-2030).
    * **Output:** Shows typos like `"3333-12-01"` (placeholder) or `"1018-04-10"` (likely 2018).
    * **Benefit:** Allows us to build "Smart Fix" logic to correct typos instead of deleting them.

4.  **Reporting:**
    * It saves a summary log to `deep_dive_log.txt`.
    * It saves detailed CSV samples of the bad data to the `00_data_issues/` folder for manual inspection.

**Why use it?**
While the first script tells us *that* there is a problem, this script tells us *what* the problem is. It provides the evidence needed to write the final `clean_aact_final.py` script with confidence.

In [ ]:
import pandas as pd
import csv
import os
import sys

# --- CONFIGURATION ---
DATA_DIR = "00_data"
OUTPUT_DIR = "00_data_issues"  # We will save error samples here
LOG_FILE = "deep_dive_log.txt"
DELIMITER = "|"

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Buffer for the log file
log_buffer = []

def log(message):
    """Prints to console and saves to buffer."""
    print(message)
    log_buffer.append(message)

def save_log():
    """Writes the full log to a file."""
    with open(LOG_FILE, "w", encoding="utf-8") as f:
        f.write("\n".join(log_buffer))
    print(f"\n[DONE] Diagnostic log saved to: {LOG_FILE}")

# --- 1. STRUCTURAL INVESTIGATOR (The "Broken Row" Finder) ---
def investigate_broken_rows(filename, problematic_lines):
    """
    Reads specific line numbers to see WHY they are broken.
    problematic_lines: list of integers (0-indexed line numbers)
    """
    file_path = os.path.join(DATA_DIR, filename)
    log(f"\n--- INVESTIGATION: Broken Rows in '{filename}' ---")

    if not os.path.exists(file_path):
        log(f"  [Error] File not found: {file_path}")
        return

    try:
        with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
            header = f.readline().strip()
            expected_cols = header.count(DELIMITER) + 1
            log(f"  Header ({expected_cols} cols): {header[:100]}...")

            # We want to peek at specific lines. Reading sequentially is safest for text files.
            current_line = 0 # Header is usually considered line 0 or we start counting after

            # Convert target lines to a set for fast lookup.
            # Note: User report said "Line 4767". In Python enumerate (0-indexed),
            # if line 1 is header, Line 4767 is index 4766. We will look around that area.
            # We will scan the whole file and check column counts again to be precise.

            bad_samples = []

            # Reset file pointer to start
            f.seek(0)

            for i, line in enumerate(f):
                cols = line.count(DELIMITER) + 1
                if cols != expected_cols:
                    # Save the bad line for analysis
                    clean_line = line.strip()
                    bad_samples.append({
                        "line_number": i + 1,
                        "actual_cols": cols,
                        "expected_cols": expected_cols,
                        "content": clean_line[:200] + "..." if len(clean_line) > 200 else clean_line
                    })
                    if len(bad_samples) >= 5: # Just get first 5 examples
                        break

            if bad_samples:
                log(f"  Found {len(bad_samples)}+ broken lines. Examples:")
                for b in bad_samples:
                    log(f"    Line {b['line_number']} (Cols {b['actual_cols']}): {b['content']}")

                # Save to CSV for user to inspect
                sample_df = pd.DataFrame(bad_samples)
                sample_path = os.path.join(OUTPUT_DIR, f"broken_rows_{filename.replace('.txt', '.csv')}")
                sample_df.to_csv(sample_path, index=False)
                log(f"  [Saved] Full bad row details to: {sample_path}")
            else:
                log("  [Info] No broken rows found (maybe they were fixed?).")

    except Exception as e:
        log(f"  [Error] Failed to read file: {e}")

# --- 2. MIXED TYPE INVESTIGATOR (The "Hidden String" Finder) ---
def investigate_mixed_types(filename, col_name):
    """
    Loads a specific column and finds values that are NOT numbers.
    """
    file_path = os.path.join(DATA_DIR, filename)
    log(f"\n--- INVESTIGATION: Mixed Types in '{filename}' (Col: {col_name}) ---")

    try:
        # Load only the specific column to save memory
        # We use strict settings to catch the garbage
        df = pd.read_csv(
            file_path,
            sep=DELIMITER,
            usecols=[col_name],
            dtype=str, # Read as string first
            quoting=csv.QUOTE_NONE,
            on_bad_lines='skip' # Skip broken rows for this check
        )

        # Try to convert to numeric
        numeric = pd.to_numeric(df[col_name], errors='coerce')

        # Find rows where conversion failed (Result is NaN) BUT original was NOT empty
        mask_bad = numeric.isna() & df[col_name].notna()
        bad_values = df[mask_bad]

        if not bad_values.empty:
            count = len(bad_values)
            log(f"  Found {count} non-numeric values in '{col_name}'.")

            # Get top frequent garbage (e.g., "10,000" vs "unknown")
            top_garbage = bad_values[col_name].value_counts().head(10)
            log(f"  Top non-numeric patterns:\n{top_garbage.to_string()}")

            # Save specific file
            out_name = f"mixed_types_{filename.replace('.txt', '')}_{col_name}.csv"
            bad_values.head(100).to_csv(os.path.join(OUTPUT_DIR, out_name), index=False)
            log(f"  [Saved] Sample of bad values to: {OUTPUT_DIR}/{out_name}")
        else:
            log("  [OK] Column appears cleanly numeric (or empty).")

    except Exception as e:
        log(f"  [Error] Could not check mixed types: {e}")

# --- 3. UNPARSEABLE DATE FINDER ---
def investigate_bad_dates(filename, col_name):
    """
    Finds dates like '3333-12-01' or 'Pending'.
    """
    file_path = os.path.join(DATA_DIR, filename)
    log(f"\n--- INVESTIGATION: Weird Dates in '{filename}' (Col: {col_name}) ---")

    try:
        df = pd.read_csv(
            file_path,
            sep=DELIMITER,
            usecols=[col_name],
            dtype=str,
            quoting=csv.QUOTE_NONE,
            on_bad_lines='skip'
        )

        # Try to convert
        dates = pd.to_datetime(df[col_name], errors='coerce')

        # Find failures
        mask_bad = dates.isna() & df[col_name].notna()
        bad_values = df[mask_bad]

        if not bad_values.empty:
            log(f"  Found {len(bad_values)} unparseable dates.")
            log(f"  Examples: {bad_values[col_name].unique()[:10]}")
        else:
            # Check for logical outliers (e.g. year 3000)
            # We filter for valid dates that are way in the future
            valid_dates = dates.dropna()
            future_mask = valid_dates > pd.Timestamp("2030-01-01")
            weird_future = valid_dates[future_mask]

            if not weird_future.empty:
                log(f"  Found {len(weird_future)} dates far in the future (Logical Errors).")
                log(f"  Examples: {weird_future.head().astype(str).tolist()}")
            else:
                log("  [OK] Dates look technically valid.")

    except Exception as e:
        log(f"  [Error] Checking dates: {e}")


# --- EXECUTION: TARGETING THE ISSUES FROM YOUR REPORT ---

log("STARTING DEEP DIVE DIAGNOSTIC based on 'data_quality_report.txt'")

# 1. Structure Issues (Broken Rows)
# responsible_parties.txt had 31 broken rows
investigate_broken_rows("responsible_parties.txt", [])
# facilities.txt had 1448 broken rows (Major issue)
investigate_broken_rows("facilities.txt", [])
# studies.txt had 55 broken rows (Critical table)
investigate_broken_rows("studies.txt", [])
# outcome_measurements.txt had 704 broken rows
investigate_broken_rows("outcome_measurements.txt", [])

# 2. Mixed Data Types (Numbers vs Text)
# baseline_measurements.txt -> param_value (text like '10,366')
investigate_mixed_types("baseline_measurements.txt", "param_value")
investigate_mixed_types("baseline_measurements.txt", "dispersion_value")

# outcome_measurements.txt -> dispersion_value
investigate_mixed_types("outcome_measurements.txt", "dispersion_value")

# facility_contacts.txt -> phone_extension (text like 'extension 1')
investigate_mixed_types("facility_contacts.txt", "phone_extension")

# 3. Date Issues
# outcomes.txt -> anticipated_posting_date (Found '3333-12-01')
investigate_bad_dates("outcomes.txt", "anticipated_posting_date")
# provided_documents.txt -> document_date (Found '1018-04-10')
investigate_bad_dates("provided_documents.txt", "document_date")

save_log()

STARTING DEEP DIVE DIAGNOSTIC based on 'data_quality_report.txt'

--- INVESTIGATION: Broken Rows in 'responsible_parties.txt' ---
  Header (8 cols): id|nct_id|responsible_party_type|name|title|organization|affiliation|old_name_title...
  Found 5+ broken lines. Examples:
    Line 4767 (Cols 9): 144841503|NCT03862352|PRINCIPAL_INVESTIGATOR|Dr Thomas J Ford|"Clinical Research Fellow | Honorary Fellow (Interventional Cardiology)"||NHS National Waiting Times Centre Board|
    Line 30129 (Cols 9): 144851281|NCT02476669|PRINCIPAL_INVESTIGATOR|Dr. Victor H.F. Lee|"Clinical Assistant |Professor"||The University of Hong Kong|
    Line 43702 (Cols 9): 144859094|NCT07108062|PRINCIPAL_INVESTIGATOR|Mohamed Abdelaziz Emam|"Lecturer assistant at of Basic Sciences Department , Faculty of Physical Therapy, Kafrelsheikh Univerisity, Kafrelsheikh 33511, Egy...
    Line 49365 (Cols 9): 144862701|NCT05772689|PRINCIPAL_INVESTIGATOR|Fayron Epps|"Associate Professor | Director of Community & Research Engagemen

# Numeric Scale Check Script: Explained

**Purpose:** This script acts as a diagnostic tool to understand the true nature of "mixed" numeric columns before we clean them. It specifically helps us decide whether percentages like `57.5%` should be treated as `57.5` (0-100 scale) or `0.575` (0-1 scale).

**What it does (Step-by-Step):**

1.  **Loads Specific Columns:** It reads only the target column (e.g., `param_value`) from the specified file, using safe loading parameters to avoid crashes.

2.  **Isolates Pure Numbers:**
    * It uses regex to find rows that contain *only* digits and decimals (e.g., `12.5`, `100`).
    * It calculates statistics (Min, Max, Mean) on these pure numbers to establish a "baseline scale."
    * *Example:* If the max pure number is `10,000`, the scale is clearly not 0-1.

3.  **Isolates Percentages:**
    * It finds rows containing the `%` symbol.
    * It strips the symbol and converts the remaining text to numbers to see their range.
    * *Example:* `57.5%` -> `57.5`.

4.  **Logic & Verdict:**
    * It compares the "Pure Number" scale vs. the "Percentage" scale.
    * **Verdict Logic:**
        * If pure numbers are small (0-1) and percentages are large (0-100), it suggests we should **DIVIDE** by 100.
        * If pure numbers are large (>1) and percentages are large (0-100), it suggests we should just **STRIP** the symbol.

**Why use it?**
Blindly stripping `%` or blindly dividing by 100 can corrupt data. This script gives us the mathematical proof needed to choose the correct cleaning strategy for our final dataset.

In [ ]:
import pandas as pd
import csv
import os

# --- CONFIGURATION ---
DATA_DIR = "00_data"
# Using your proven loading params
LOAD_PARAMS = {
    "sep": "|",
    "dtype": str,
    "quotechar": '"',
    "quoting": csv.QUOTE_MINIMAL,
    "low_memory": False,
    "on_bad_lines": "warn"
}

def check_numeric_scale(filename, col_name):
    path = os.path.join(DATA_DIR, filename)
    if not os.path.exists(path): return

    print(f"\n--- Checking Scale: {filename} [{col_name}] ---")

    try:
        df = pd.read_csv(path, usecols=[col_name], **LOAD_PARAMS)

        # 1. Isolate Pure Numbers
        # (Rows that are just digits and dots, no text/symbols)
        pure_numbers = df[col_name][df[col_name].astype(str).str.match(r'^-?\d+\.?\d*$')]
        pure_floats = pd.to_numeric(pure_numbers)

        if pure_floats.empty:
            print("  No pure numbers found to compare against.")
            return

        print(f"  Pure Number Stats (Sample size: {len(pure_floats)}):")
        print(f"    Min: {pure_floats.min()}")
        print(f"    Max: {pure_floats.max()}")
        print(f"    Mean: {pure_floats.mean():.2f}")

        # 2. Isolate Percentage Strings
        percent_strings = df[col_name][df[col_name].astype(str).str.contains(r'%')]

        if percent_strings.empty:
            print("  No '%' signs found in this column.")
        else:
            print(f"  Found {len(percent_strings)} rows with '%'.")
            print(f"    Examples: {percent_strings.head().tolist()}")

            # Logic Check
            clean_percents = pd.to_numeric(percent_strings.str.replace('%', ''), errors='coerce')
            print(f"    If we just strip '%', the range is: {clean_percents.min()} to {clean_percents.max()}")

            # Decision Helper
            if pure_floats.max() <= 1.0 and clean_percents.mean() > 1.0:
                print("\n  [VERDICT] Raw numbers are likely 0-1 (Decimals). You SHOULD divide % by 100.")
            elif pure_floats.mean() > 1.0:
                print("\n  [VERDICT] Raw numbers are likely 0-100 (Integers). You should STRIP % (Don't divide).")
            else:
                print("\n  [VERDICT] Scale is ambiguous. Manual review needed.")

    except Exception as e:
        print(f"Error: {e}")

# Check the problematic columns identified in logs
check_numeric_scale("outcome_measurements.txt", "dispersion_upper_limit_raw")
check_numeric_scale("outcome_measurements.txt", "param_value")
check_numeric_scale("baseline_measurements.txt", "param_value")


--- Checking Scale: outcome_measurements.txt [dispersion_upper_limit_raw] ---
  Pure Number Stats (Sample size: 759279):
    Min: -5860000.0
    Max: 785070000000000.0
    Mean: 2021412878.24
  Found 11 rows with '%'.
    Examples: ['57.5%', '48%', '9.8%', '100%', '99.4%']
    If we just strip '%', the range is: 0.7 to 100.0

  [VERDICT] Raw numbers are likely 0-100 (Integers). You should STRIP % (Don't divide).

--- Checking Scale: outcome_measurements.txt [param_value] ---
  Pure Number Stats (Sample size: 4553587):
    Min: -408000000.0
    Max: 875769282253977.0
    Mean: 423983377.92
  No '%' signs found in this column.

--- Checking Scale: baseline_measurements.txt [param_value] ---
  Pure Number Stats (Sample size: 2712770):
    Min: -663.0
    Max: 13920879614.0
    Mean: 13802.03
  No '%' signs found in this column.


# 🏭 Master Data Conversion Pipeline: Explained

**Purpose:** This is the final "production" script for data preparation. It processes every single file in our raw data folder (`00_data`) and converts them into a standardized, clean format ready for Machine Learning.

**What it does (Step-by-Step):**

1.  **Iterates through EVERYTHING:** It loops through all 50+ `.txt` files in the source folder. This ensures no part of the database is left behind in an old format.

2.  **Correct Loading (The "Safe Fix"):**
    * It applies the proven structural fix (`quotechar='"'`) to every file.
    * This guarantees that files with "pipes inside quotes" (like `facilities.txt`) are loaded correctly without breaking rows.

3.  **Surgical Cleaning (The "Smart Logic"):**
    * It checks a **Special Rules Registry**.
    * If a file is known to be "dirty" (e.g., `baseline_measurements.txt`), it applies specific cleaning functions:
        * **Numbers:** Removes commas, spaces, and `%` signs to create pure numbers.
        * **Dates:** Fixes typos (e.g., `1018` -> `2018`) and filters impossible years.
    * If a file is "standard," it skips deep cleaning to preserve data integrity.

4.  **Standard Hygiene:**
    * For *every* file, it trims invisible whitespace from text columns (e.g., `" Diabetes "` -> `"Diabetes"`).

5.  **Standard Export:**
    * It saves every file as a **Pipe-Delimited Text File (`.txt`)**.
    * **Crucial Feature:** It uses `quoting=csv.QUOTE_MINIMAL`. This ensures that if any field contains a pipe `|`, it is wrapped in quotes so it doesn't break the file structure again.

**Why use it?**
This script gives us a **single, unified folder (`00_data_ml_ready`)** where every file is structurally sound, clean, and formatted exactly the same way. This is the "Gold Standard" dataset for our project.

In [ ]:
import pandas as pd
import csv
import os
import numpy as np

# --- CONFIGURATION ---
DATA_DIR = "00_data"
OUTPUT_DIR = "00_data_ml_ready"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 1. STRUCTURAL SETTINGS (Proven Correct)
AACT_LOAD_PARAMS = {
    "sep": "|",
    "dtype": str,                 # Read all as string first
    "quotechar": '"',             # Handles pipes inside quotes
    "quoting": csv.QUOTE_MINIMAL,
    "low_memory": False,
    "on_bad_lines": "warn"
}

# --- CLEANING FUNCTIONS (Matching v3 Logic) ---
def clean_numeric_column(series):
    """
    Surgical Cleaning:
    - Removes spaces, commas, <, >
    - Strips % (Keeps 0-100 scale)
    - Returns floats (or NaN for text/ranges)
    """
    clean = series.str.replace(r'\s+', '', regex=True)
    clean = clean.str.replace(r'[,<>]', '', regex=True)
    clean = clean.str.replace('%', '')
    return pd.to_numeric(clean, errors='coerce')

def clean_date_column(series):
    """
    Smart Date Cleaning:
    - Fixes '10xx' typos (1018 -> 2018)
    - Filters 1900-2100 (Keeps future trials)
    """
    series = series.astype(str).str.replace(r'^10(\d{2}-\d{2}-\d{2})', r'20\1', regex=True)
    dates = pd.to_datetime(series, errors='coerce')
    mask_valid = (dates.dt.year >= 1900) & (dates.dt.year <= 2100)
    return dates.where(mask_valid, pd.NaT)

# --- SPECIAL RULES REGISTRY ---
# Files NOT in this list will get a standard "Safe Conversion"
SPECIAL_CONFIG = {
    "studies.txt": {
        "date_cols": ["study_first_submitted_date", "results_first_submitted_date", "completion_date", "start_date"]
    },
    "baseline_measurements.txt": {
        "numeric_cols": ["param_value", "dispersion_value"]
    },
    "outcome_measurements.txt": {
        "numeric_cols": ["param_value", "dispersion_value", "dispersion_upper_limit_raw", "dispersion_lower_limit_raw"]
    },
    "outcome_analyses.txt": {
        "numeric_cols": ["ci_upper_limit_raw", "ci_lower_limit_raw", "p_value"]
    },
    "outcomes.txt": {
        "date_cols": ["anticipated_posting_date"]
    },
    "provided_documents.txt": {
        "date_cols": ["document_date"]
    }
}

def process_file(filename):
    in_path = os.path.join(DATA_DIR, filename)
    # OUTPUT CHANGE: Saving as .txt to match original format
    out_path = os.path.join(OUTPUT_DIR, filename)

    # Skip if not a text file or if it's a report file
    if not filename.endswith(".txt") or "report" in filename: return

    print(f"Processing: {filename}...", end=" ")

    try:
        # 1. LOAD (Standard Safe Load)
        df = pd.read_csv(in_path, **AACT_LOAD_PARAMS)

        # 2. STANDARD HYGIENE (Apply to ALL files)
        # Strip invisible whitespace from all text columns
        obj_cols = df.select_dtypes(include=['object']).columns
        for col in obj_cols:
            df[col] = df[col].str.strip()

        # 3. SPECIAL CLEANING (Only for specific files)
        if filename in SPECIAL_CONFIG:
            rules = SPECIAL_CONFIG[filename]

            # Apply Numeric Rules
            for col in rules.get("numeric_cols", []):
                if col in df.columns:
                    df[col] = clean_numeric_column(df[col])

            # Apply Date Rules
            for col in rules.get("date_cols", []):
                if col in df.columns:
                    df[col] = clean_date_column(df[col])

            print(f"[Special Cleaning Applied] -> ", end="")
        else:
            print(f"[Standard Cleaning] -> ", end="")

        # 4. EXPORT (Clean Pipe-Delimited TXT)
        # sep='|' keeps it as a pipe file.
        # quoting=csv.QUOTE_MINIMAL is crucial: it puts quotes around fields
        # that contain pipes (fixing the original structural bug).
        df.to_csv(out_path, index=False, sep='|', quoting=csv.QUOTE_MINIMAL)
        print(f"Saved to {out_path} ({len(df)} rows)")

    except Exception as e:
        print(f"\n  [ERROR] Failed: {e}")

# --- MAIN EXECUTION ---
print(f"Starting Batch Cleaning from '{DATA_DIR}' to '{OUTPUT_DIR}'...\n")

files = sorted(os.listdir(DATA_DIR))
txt_files = [f for f in files if f.endswith(".txt")]

for i, filename in enumerate(txt_files):
    process_file(filename)

print(f"\n[DONE] {len(txt_files)} files cleaned and saved successfully.")

Starting Batch Cleaning from '00_data' to '00_data_ml_ready'...

Processing: baseline_counts.txt... [Standard Cleaning] -> Saved to 00_data_ml_ready/baseline_counts.txt (224320 rows)
Processing: baseline_measurements.txt... [Special Cleaning Applied] -> Saved to 00_data_ml_ready/baseline_measurements.txt (2720829 rows)
Processing: brief_summaries.txt... [Standard Cleaning] -> Saved to 00_data_ml_ready/brief_summaries.txt (558028 rows)
Processing: browse_conditions.txt... [Standard Cleaning] -> Saved to 00_data_ml_ready/browse_conditions.txt (4084129 rows)
Processing: browse_interventions.txt... [Standard Cleaning] -> Saved to 00_data_ml_ready/browse_interventions.txt (2432357 rows)
Processing: calculated_values.txt... [Standard Cleaning] -> Saved to 00_data_ml_ready/calculated_values.txt (558973 rows)
Processing: central_contacts.txt... [Standard Cleaning] -> Saved to 00_data_ml_ready/central_contacts.txt (211164 rows)
Processing: conditions.txt... [Standard Cleaning] -> Saved to 00_da